# Dealing with Data Spring 2019 (INFO-GB.2346.30)
## Homework #7 - SOLVED VERISON

### **Problem 1 (4 points):**
Use our Class Project SQLite3 database through Python to find the zip codes who each had at least 10,000 residents earning 100,000 USD or more annually during 2016. Print your results in ascending order by zipcode, for example: 10001, 10002.

In [11]:
## solution
import sqlite3, os
os.chdir(r'C:\Users\colling\!dwd_spring2019\classes\class8')

# setup location variables for the database
db_location = r'nyc_film_db_final.db'

# Create the connection
db_connect = sqlite3.connect(db_location)

# create SQL statement
sql_cmd = '''
SELECT zipcode 
FROM irs_nyc_tax_returns 
WHERE agi_map_id IN (5,6) AND year = "2016" 
GROUP BY zipcode 
HAVING SUM(return_count) > 10000
ORDER BY zipcode ASC;'''

# create cursor, execute query using cursor, retrieve results & headers and close db
db_cursor = db_connect.cursor()
db_cursor.execute(sql_cmd)
results_data = db_cursor.fetchall() # retrieve results
db_cursor.close()
db_connect.close()

# pretty up results
print_results = []
for result in results_data:
    print_results.append(result[0])
    print_results.sort()  # note: this is only required if the SQL statement did NOT include ORDER BY
    
# print
print(print_results)

['10003', '10011', '10016', '10019', '10021', '10022', '10023', '10024', '10025', '10028', '10128', '10583', '11201', '11215', '11375']


### **Problem 2 (6 points):**
The NYC film permit's database stores datetime values as "12/09/2018 07:00:00 AM" and "12/09/2018 07:00:00 PM" but SQLite3 will NOT recognize this value as a date because it requires that datetime values are formatted as "2018-12-09 07:00" and "2018-12-09 19:00", respectively (note the use of a 24-hour time in the second example). Your assignment is to:
* Create a UDF that will take the NYC film permit's datetime value input and user regular expressions to transform it into the SQLite3 compliant datetime value. 
* Prove it works by running the UDF against at least test_input_1 and test_input_2 below and comparing them both to their respective outputs. Full credit will be given for UDFs that handle all datetimes correctly beyond the provided test inputs.

In [74]:
## here are test inputs and expected outputs to test your code
test_input_1  = "12/09/2018 07:00:00 AM"
expected_output_1 = "2018-12-09 07:00"

test_input_2  = "02/13/2015 07:00:00 PM"
expected_output_2 = "2015-02-13 19:00"

In [96]:
# solution
def date_convert(input_date):
    import re, sys
    
    # define regex
    date_regex = re.compile(r'''
        (\d{2})/  #group 1# month
        (\d{2})/  #2# day
        (\d{4})   #3# year
        \s{1}     ## space delimiter
        (\d{2}):  #4# hours
        (\d{2}):  #5# minutes
        (\d{2})   #6# seconds
        \s{1}     ## space delimiter
        (AM|PM)   #7# meridiem indicator
   ''',re.VERBOSE) 

    # grab groups
    match = date_regex.search(input_date)
    
    # if match is empty, return error
    if match == None:
        print("bad input!")
        sys.exit(1)
    
    # build new date, item by item
    input_month = str(match.group(1))
    input_day = str(match.group(2))
    input_year = str(match.group(3))
    input_hour = str(match.group(4))
    input_minute = str(match.group(5))
    input_seconds = str(match.group(6))
    input_meridiem = str(match.group(7))
    
    # convert AM/PM hours to 24-hr format
    if (input_meridiem.lower() == "pm" and int(input_hour) < 12):
        input_hour = str(int(input_hour) + 12)
    elif (input_meridiem.lower() == "am" and int(input_hour) == 12):
        input_hour = "00"

    
    return input_year + "-" + input_month + "-" + input_day + " " + input_hour + ":" + input_minute

In [97]:
# test provided inputs
test_input_1  = "12/09/2018 07:00:00 AM"
expected_output_1 = "2018-12-09 07:00"
print("test 1 ==>",expected_output_1 == date_convert(test_input_1))

test_input_2  = "02/13/2015 07:00:00 PM"
expected_output_2 = "2015-02-13 19:00"
print("test 2 ==>",expected_output_2 == date_convert(test_input_2))

# test edge case of midnight
print("converting 02/13/2015 12:00:00 AM ==>",date_convert("02/13/2015 12:00:00 AM"))

test 1 ==> True
test 2 ==> True
converting 02/13/2015 12:00:00 AM ==> 2015-02-13 00:00
